In [1]:
import pickle
import random
import logging
import time
import pickle
import json
import os
import pathlib
from os.path import dirname, abspath
from rdkit import Chem

import numpy as np
import torch
from rdkit import RDLogger
from rdkit.Chem import AllChem
from rdkit import Chem

import rootutils
root_dir = rootutils.setup_root(".",
                               indicator=".project-root",
                               pythonpath=True)

In [2]:
# load /home/ubuntu/recursiveLLM/data/routes_possible_test_hard.pkl
with open('/home/ubuntu/recursiveLLM/data/routes_possible_test_hard.pkl', 'rb') as f:
    routes = pickle.load(f)
    # print the keys of the dictionary

In [3]:
target_mols = [Chem.CanonSmiles(route[0].split('>')[0]) for route in routes]
num_targets = len(target_mols)
target_mols_id = list(range(num_targets))
max_len = 16 # for USPTO test molecules

In [4]:
len(target_mols)

190

In [11]:
random.seed(21)
# sample 10 molecules from target_mols with seed 21
sampled_mols = random.sample(target_mols, 190)
# print the sampled molecules
# save sampled_mols to csv file
import pandas as pd
df = pd.DataFrame(sampled_mols, columns=['smiles'])
df.to_csv(f"{root_dir}/data/USPTO_190.csv", index=False)


In [7]:
len(sampled_mols)

190

In [ ]:
# save sampled_mols to csv file

['Cn1nccc1[C@]1(O)CCCC[C@H]1O',
 'CC1=NC2(N=C1N)c1cc(-c3cc(Cl)cc(C#N)c3)ccc1CCC21CC1',
 'CCC/C=C/[C@H]1CC[C@H](C(CO)CO)CC1',
 'COCCCc1cc(CN(C(=O)[C@H]2CN(C(=O)OC(C)(C)C)CC[C@@H]2c2ccc(OCCOc3c(Cl)cc(C)cc3Cl)cc2)C2CC2)cc(OCC(C)(C)O)c1',
 'C=CCC1C=C(C)CC(C)CC(OC)C2OC(O)(C(=O)C(=O)N3CCCCC3C(=O)OC(C(C)=CC3CCC(O)C(OC)C3)C(C)C=CC1O)C(C)CC2OC',
 'CC(=O)NC[C@H]1CC[C@@H](C(=O)Nc2cccc(OC(F)(F)F)c2)N1',
 'COC(=O)c1ccc2c(c1)C=CC(=C(Cl)Cl)CO2',
 'Cc1ccc(C(C)(C)C)cc1S[C@@H]1O[C@H](CO)[C@@H](O)[C@H](O)[C@H]1O',
 'COC(=O)[C@@H]1CCCC2(CCCCC2)[C@H]1O',
 'CC(C)S(=O)(=O)N[C@H]1CCOC[C@H]1c1ccc(I)cc1',
 'CN1CC[C@@H](c2c(O)cc(O)c3c(=O)cc(-c4ccc(C(F)(F)F)cc4)oc23)[C@@H]1CO',
 'CC#CCn1c(Br)nc(C=O)c1C(=O)OC',
 'Cc1ccn2cc(CO)n(-c3ccccc3)c(=O)c12',
 'CC(C)(CO)n1c(CO)nc2cnc(Br)cc21',
 'C[C@H](c1ccccc1)N1C[C@]2(C(=O)OC(C)(C)C)C=CC[C@@H]2C1=S',
 'CCOC(=O)c1nc(N2CC[C@H](NC(=O)c3nc(C(F)(F)F)c(CC)[nH]3)[C@H](OC)C2)sc1C',
 'COC[C@H](C)COCc1ccc([C@@]2(O)CCNC[C@@H]2c2noc(-c3ccccc3CCNC(C)=O)c2Br)cc1',
 'COCCCc1cc(CN(C(=O)[C

In [8]:
# Sampled paths
sampled_paths = [route for route in routes if route[0].split('>')[0] in sampled_mols]
# print the sampled paths
# print(sampled_paths)
# save sampled_paths to pickle file
# with open('/home/ubuntu/recursiveLLM/data/USPTO_190_sample_paths.pkl', 'wb') as f:
#     pickle.dump(sampled_paths, f)
# save sampled_paths to csv file



In [ ]:
gt_paths = []
for idx,sample in enumerate(sampled_paths):
    temp = {}
    for idx2,step in enumerate(sample):
        temp_step = {}
        inp = step.split('>')[0]
        out = step.split('>')[2].split('.')
        # convert molecule to canonical smiles
        inp = Chem.CanonSmiles(inp)
        out = [Chem.CanonSmiles(i) for i in out]
        temp_step['input'] = inp
        temp_step['output'] = out
        temp[idx2] = temp_step
    # print(temp)
    # print('--------------------------------')
    gt_paths.append(temp)


{0: {'input': 'C[C@H](c1ccccc1)N1C[C@]2(C(=O)OC(C)(C)C)C=CC[C@@H]2C1=S', 'output': ['C[C@H](c1ccccc1)N1C[C@]2(C(=O)OC(C)(C)C)C=CC[C@@H]2C1=O', 'COc1ccc(P2(=S)SP(=S)(c3ccc(OC)cc3)S2)cc1']}, 1: {'input': 'C[C@H](c1ccccc1)N1C[C@]2(C(=O)OC(C)(C)C)C=CC[C@@H]2C1=O', 'output': ['C=CC[C@@H]1C(=O)N([C@H](C)c2ccccc2)C[C@@]1(C=C)C(=O)OC(C)(C)C']}, 2: {'input': 'C=CC[C@@H]1C(=O)N([C@H](C)c2ccccc2)C[C@@]1(C=C)C(=O)OC(C)(C)C', 'output': ['C=C[C@]1(C(=O)OC(C)(C)C)CC(=O)N([C@H](C)c2ccccc2)C1', 'C=CCBr']}, 3: {'input': 'C=C[C@]1(C(=O)OC(C)(C)C)CC(=O)N([C@H](C)c2ccccc2)C1', 'output': ['C[C@H](c1ccccc1)N1C[C@@](C=O)(C(=O)OC(C)(C)C)CC1=O', '[Li]CCCC']}, 4: {'input': 'C[C@H](c1ccccc1)N1C[C@@](C=O)(C(=O)OC(C)(C)C)CC1=O', 'output': ['C[C@H](c1ccccc1)N1C[C@@](CO)(C(=O)OC(C)(C)C)CC1=O']}, 5: {'input': 'C[C@H](c1ccccc1)N1C[C@@](CO)(C(=O)OC(C)(C)C)CC1=O', 'output': ['C[C@H](c1ccccc1)N1CC(C(=O)OC(C)(C)C)CC1=O', 'O=C(O)CC(O)(CC(=O)O)C(=O)O']}, 6: {'input': 'C[C@H](c1ccccc1)N1CC(C(=O)OC(C)(C)C)CC1=O', 'output': ['C

{0: {'input': 'Cn1oc(=O)nc1/C(=N\\OCc1cccc(NC(=O)OCCc2ccccc2)n1)c1ccccc1', 'output': ['Cn1oc(=O)nc1/C(=N\\OCc1cccc(N)n1)c1ccccc1', 'O=C(Cl)OCCc1ccccc1']}, 1: {'input': 'Cn1oc(=O)nc1/C(=N\\OCc1cccc(N)n1)c1ccccc1', 'output': ['Cn1oc(=O)nc1/C(=N\\OCc1cccc(NC(=O)OC(C)(C)C)n1)c1ccccc1']}, 2: {'input': 'Cn1oc(=O)nc1/C(=N\\OCc1cccc(NC(=O)OC(C)(C)C)n1)c1ccccc1', 'output': ['O=C(n1ccnc1)n1ccnc1', 'CN(O)C(=N)/C(=N\\OCc1cccc(NC(=O)OC(C)(C)C)n1)c1ccccc1']}, 3: {'input': 'CN(O)C(=N)/C(=N\\OCc1cccc(NC(=O)OC(C)(C)C)n1)c1ccccc1', 'output': ['CC(C)O', 'O=C([O-])[O-]', 'CNO', 'CC(C)(C)OC(=O)Nc1cccc(CO/N=C(\\C#N)c2ccccc2)n1']}, 4: {'input': 'CC(C)(C)OC(=O)Nc1cccc(CO/N=C(\\C#N)c2ccccc2)n1', 'output': ['CC(C)(C)OC(=O)Nc1cccc(CCl)n1', 'N#C/C(=N\\O)c1ccccc1']}}
--------------------------------
{0: {'input': 'COc1ccccc1C[C@H](C[C@H](O[Si](C)(C)C(C)(C)C)[C@H](Cc1ccccc1)NC(=O)OC(C)(C)C)C(=O)O', 'output': ['CC(C)(C)[Si](C)(C)Cl', 'COc1ccccc1C[C@H](C[C@H](O)[C@H](Cc1ccccc1)NC(=O)OC(C)(C)C)C(=O)O']}, 1: {'input': 

## Retro*

In [11]:
with open('/home/ubuntu/recursiveLLM/retro_star/retro_star/results/plan_new_cleaned.pkl', 'rb') as f:
    result_retro_star = pickle.load(f)

In [59]:
retro_star_paths = []
for idx,sample in enumerate(result_retro_star['routes']):
    temp = {}
    for idx2,step in enumerate(sample):
        temp_step = {}
        inp = step[0]
        out = step[1].split('.')
        # convert molecule to canonical smiles
        inp = Chem.CanonSmiles(inp)
        out = [Chem.CanonSmiles(i) for i in out]
        temp_step['input'] = inp
        temp_step['output'] = out
        temp[idx2] = temp_step
    print(temp)
    print('--------------------------------')
    retro_star_paths.append(temp)

{0: {'input': 'Cn1nccc1[C@]1(O)CCCC[C@H]1O', 'output': ['O', 'Cn1nccc1C1=CCCCC1', 'CS(N)(=O)=O']}, 1: {'input': 'Cn1nccc1C1=CCCCC1', 'output': ['Cn1nccc1C1(O)CCCCC1']}}
--------------------------------
{0: {'input': 'CN1CC[C@@H](c2c(O)cc(O)c3c(=O)cc(-c4ccc(C(F)(F)F)cc4)oc23)[C@@H]1CO', 'output': ['COc1cc(OC)c2c(=O)cc(-c3ccc(C(F)(F)F)cc3)oc2c1[C@@H]1CCN(C)[C@H]1CO']}, 1: {'input': 'COc1cc(OC)c2c(=O)cc(-c3ccc(C(F)(F)F)cc3)oc2c1[C@@H]1CCN(C)[C@H]1CO', 'output': ['COc1cc(OC)c([C@@H]2CCN(C)[C@H]2CO)c(O)c1C(C)=O', 'COC(=O)c1ccc(C(F)(F)F)cc1']}, 2: {'input': 'COc1cc(OC)c([C@@H]2CCN(C)[C@H]2CO)c(O)c1C(C)=O', 'output': ['CC(=O)OC(C)=O', 'COc1cc(OC)c([C@@H]2CCN(C)[C@H]2CO)c(OC)c1']}, 3: {'input': 'CC(=O)OC(C)=O', 'output': ['O=C1OC(=O)c2c(Cl)cccc21']}, 4: {'input': 'COc1cc(OC)c([C@@H]2CCN(C)[C@H]2CO)c(OC)c1', 'output': ['COc1cc(OC)c([C@@H]2CCN(C)[C@H]2COC(C)=O)c(OC)c1']}, 5: {'input': 'COc1cc(OC)c([C@@H]2CCN(C)[C@H]2COC(C)=O)c(OC)c1', 'output': ['CC(=O)[O-]', 'COc1cc(OC)c([C@@H]2CCN(C)C[C@H]2O)c

In [ ]:
# See which molecules dont have paths in gt_paths'
gt_paths_mols = [i[0]['input'] for idx,i in enumerate(gt_paths) if i is not None]
retro_star_paths_mols = [i[0]['input'] for idx,i in enumerate(retro_star_paths) if i is not None]
# Find molecules in gt_paths_mols that are not in retro_star_paths_mols and insert None to retro_star_paths
retro_star_paths_all = []
i = 0
for idx, path in enumerate(gt_paths_mols):
    if path in retro_star_paths_mols:
        retro_star_paths_all.append(retro_star_paths[idx-i])
    else:
        retro_star_paths_all.append(None)
        i += 1


In [ ]:
# retro_star_paths = retro_star_paths[:7] + [None, None] + retro_star_paths[8:]

In [81]:
# Compare gt_paths and retro_star_paths_all
failed_paths = []
for idx,sample in enumerate(gt_paths):
    if retro_star_paths_all[idx] is None:
        print(f"Sample {idx} is None")
        failed_paths.append(idx)
        continue
    for idx2,step in sample.items():
        # print(step['input']== retro_star_paths_all[idx][idx2]['input'])
        if step['input'] != retro_star_paths_all[idx][idx2]['input']:
            print(f"Mismatch at step {idx2} for sample {idx} input: {step['input']} , gt: {step['input']}, retro_star: {retro_star_paths_all[idx][idx2]['input']}")
            failed_paths.append(idx)
            break
        elif sorted(step['output']) != sorted(retro_star_paths_all[idx][idx2]['output']):
            print(f"Mismatch at step {idx2} for sample {idx}, gt: {step['output']}, retro_star: {retro_star_paths_all[idx][idx2]['output']}")
            failed_paths.append(idx)
            break
        
print('Retro* All Correct: ', (1-len(failed_paths)/len(gt_paths))*100, "%")

Mismatch at step 3 for sample 1 input: COc1cc(OC)c([C@@H]2CCN(C)[C@H]2CO)c(OC)c1 , gt: COc1cc(OC)c([C@@H]2CCN(C)[C@H]2CO)c(OC)c1, retro_star: CC(=O)OC(C)=O
Mismatch at step 1 for sample 3, gt: ['NC[C@H]1CC[C@@H](C(=O)Nc2cccc(OC(F)(F)F)c2)N1C(=O)OCc1ccccc1', 'CC(=O)Cl'], retro_star: ['Nc1cccc(OC(F)(F)F)c1', 'CC(=O)NC[C@H]1CC[C@@H](C(=O)O)N1C(=O)OCc1ccccc1']
Mismatch at step 0 for sample 5, gt: ['COC(=O)C1CCCC2(CCCCC2)C1=O'], retro_star: ['CO', 'O=C(O)[C@@H]1CCCC2(CCCCC2)[C@H]1O']
Mismatch at step 1 for sample 6, gt: ['COC(=O)c1ccc2c(c1)C=CC(O)(CO)CO2'], retro_star: ['O=C1C=Cc2cc(C(=O)O)ccc2OC1', 'CO']
Sample 7 is None
Sample 8 is None
Mismatch at step 4 for sample 9, gt: ['CCCC#CC1CCC2(CC1)OCCO2'], retro_star: ['O=CC1CCC(=O)CC1', 'CCCC[P+](c1ccccc1)(c1ccccc1)c1ccccc1']
Retro* All Correct:  30.000000000000004 %


In [9]:
# load retro_star/retro_star/results/plan.pkl

# with open('/home/ubuntu/recursiveLLM/retro_star/retro_star/results/plan.pkl', 'rb') as f:
#     result_retro_star = pickle.load(f)


In [18]:
# cleaned_rt = []
# for rt in result_retro_star['routes']:
#     if rt is not None:
#         temp = []
#         for r in rt:
#             temp.append([r.split('>')[0], r.split('>')[2]])
#         cleaned_rt.append(temp)
# result_retro_star['routes'] = cleaned_rt
# save result_retro_star to pickle file




In [17]:
result_retro_star['routes']
with open('/home/ubuntu/recursiveLLM/retro_star/retro_star/results/plan_new_cleaned.pkl', 'wb') as f:
    pickle.dump(result_retro_star, f)

In [12]:
# new_routes = []
# for rt in result_retro_star['routes']:
#     if rt is not None:
#         new_routes.append(rt.serialize().split('|'))
#     else:
#         new_routes.append(None)
# new_routes

[['Cn1nccc1[C@]1(O)CCCC[C@H]1O>0.9471>O.Cn1nccc1C1=CCCCC1.CS(N)(=O)=O',
  'Cn1nccc1C1=CCCCC1>0.9992>Cn1nccc1C1(O)CCCCC1'],
 ['CN1CC[C@@H](c2c(O)cc(O)c3c(=O)cc(-c4ccc(C(F)(F)F)cc4)oc23)[C@@H]1CO>0.9960>COc1cc(OC)c2c(=O)cc(-c3ccc(C(F)(F)F)cc3)oc2c1[C@@H]1CCN(C)[C@H]1CO',
  'COc1cc(OC)c2c(=O)cc(-c3ccc(C(F)(F)F)cc3)oc2c1[C@@H]1CCN(C)[C@H]1CO>0.9976>COc1cc(OC)c([C@@H]2CCN(C)[C@H]2CO)c(O)c1C(C)=O.COC(=O)c1ccc(C(F)(F)F)cc1',
  'COc1cc(OC)c([C@@H]2CCN(C)[C@H]2CO)c(O)c1C(C)=O>0.9998>CC(=O)OC(C)=O.COc1cc(OC)c([C@@H]2CCN(C)[C@H]2CO)c(OC)c1',
  'CC(=O)OC(C)=O>0.9993>O=C1OC(=O)c2c(Cl)cccc21',
  'COc1cc(OC)c([C@@H]2CCN(C)[C@H]2CO)c(OC)c1>0.0535>COc1cc(OC)c([C@@H]2CCN(C)[C@H]2COC(C)=O)c(OC)c1',
  'COc1cc(OC)c([C@@H]2CCN(C)[C@H]2COC(C)=O)c(OC)c1>0.0735>CC(=O)[O-].COc1cc(OC)c([C@@H]2CCN(C)C[C@H]2O)c(OC)c1',
  'COc1cc(OC)c([C@@H]2CCN(C)C[C@H]2O)c(OC)c1>0.0535>COc1cc(OC)c(C2=CCN(C)CC2)c(OC)c1.[OH-]'],
 ['Cc1ccc(C(C)(C)C)cc1S[C@@H]1O[C@H](CO)[C@@H](O)[C@H](O)[C@H]1O>0.0010>CC(=O)OC[C@H]1O[C@@H](Sc2cc(C(C)

In [13]:
# result_retro_star['routes'] = new_routes

In [14]:
# # save result_retro_star to pickle file
# with open('/home/ubuntu/recursiveLLM/retro_star/retro_star/results/plan_new.pkl', 'wb') as f:
#     pickle.dump(result_retro_star, f)


In [19]:
# for rt in result_retro_star['routes']:
#     if rt is not None:
#         print(rt.viz_route())

## Our system

In [38]:
def get_results_in_file(file):
    res_formatted = {}
    for step in file['steps']:
        res_formatted[int(step['step'])-1] = {
            'input': Chem.CanonSmiles(step['products'][0]['smiles']),
            'output': [Chem.CanonSmiles(i['smiles']) for i in step['reactants']]
        }
    return res_formatted, file['steps'][0]['reactants'][0]['smiles']

In [ ]:
# Read Json files from results/benchmarking/USPTO_190_sample
dir_results_main = "../results/benchmarking/USPTO_190_sample"
folder_list = ["m0p1:Pistachio_100+", "m1p0:Pistachio_100+","m0p1:USPTO", "m1p0:USPTO" ]

for fol in folder_list:
    dir_results = f"{dir_results_main}/{fol}"
    for run in range(0,1):
        try:
            dir_results = f"{dir_results}/run_{run}/"
            files = [
                f for f in os.listdir(dir_results)
                if os.path.isfile(os.path.join(dir_results, f))
            ]
            # print(dir_results)
            # print(files, len(files))
            results = {}
            files.sort()
            for file in files:
                with open(os.path.join(dir_results, file)) as f:
                    res_formatted, mol_smiles = get_results_in_file(json.load(f))
                    results[mol_smiles] = res_formatted
            # print(results)
            final_list = [" "]*len(gt_paths)
            for idx, path in enumerate(gt_paths):
                # print(f'{i}.json')
                if f'{i}.json' in results:
                    final_list.append(results[f'{i}.json'])
                else:
                    final_list.append(None)
            failed_paths = []
            for idx,sample in enumerate(gt_paths):
                if final_list[idx] is None:
                    print(f"Sample {idx} is None")
                    failed_paths.append(idx)
                    continue
                for idx2,step in sample.items():
                    # print(step['input']== final_list[idx][idx2]['input'])
                    if step['input'] != final_list[idx][idx2]['input']:
                        print(f"Mismatch at step {idx2} for sample {idx} input: {step['input']} , gt: {step['input']}, DeepRetro: {final_list[idx][idx2]['input']}")
                        failed_paths.append(idx)
                        break
                    elif sorted(step['output']) != sorted(final_list[idx][idx2]['output']):
                        print(f"Mismatch at step {idx2} for sample {idx}, gt: {step['output']}, DeepRetro: {final_list[idx][idx2]['output']}")
                        failed_paths.append(idx)
                        break
                    
            print(f'DeepRetro({fol}) All Correct: ', (1-len(failed_paths)/len(gt_paths))*100, "%")
        except:
            pass
